In this notebook we determine which of the integer slopes of the Baker-Luecke knots admit OSBs.

In [1]:
##This is code that will search for an obtuse superbase
##in an integer CM lattice. Input is a list of
##coefficients for CM lattice.

from itertools import combinations
import numpy as np
import time
import csv

def norm(v):
    return sum([z*z for z in v])

def irred_check(v,vtest):
    #Computes T=(v-vtest).vtest returns true if this does not prove
    #that v is reducible. I.e returns true if T<0 or vtest=0 or v=vtest.

    T=sum([(v[i]-vtest[i])*vtest[i] for i in range(0,len(v))])
    if T<0:
        return True

    if T==0:
        if sum([z*z for z in vtest])==0:
            return True
        if sum([(v[i]-vtest[i])*(v[i]-vtest[i]) for i in range(0,len(v))])==0:
            return True
    return False

def irred_bounds(sigma):
    #Provide bounds on the coefficients of an irreducible
    #in a changemaker lattice.
    n=len(sigma)
    vmax=[0]*n
    
    #Check if sigma is tight
    tight=False
    for i in range(1,n):
        if sigma[i]> sum([sigma[j] for j in range(0,i)]):
            tight=True
            break

    if tight:
        vmax[0]=2
    else:
        vmax[0]=1
    
    for i in range(1, len(sigma)):
        if sigma[i]==sigma[i-1]:
            vmax[i]=vmax[i-1]
        else:
            #If sigma[i] tight.
            if sigma[i]> sum([sigma[j] for j in range(0,i)]):
                vmax[i]=6+sum([vmax[j]+1 for j in range(0,i)])
                vmax[i]=min(vmax[i], sigma[i]*(sigma[i]+2))
            else:
                T=sigma[i]
                j=i-1
                while T>0:
                    j=max([k for k in range(0,j+1) if sigma[k]<=T])
                    vmax[i]=vmax[i]+vmax[j]+1
                    T-=sigma[j]
                    
    #print(vmax)
    return(vmax)

def irred_generator(sigma, vmax):
    n=len(sigma)
    v=[-vmax[i] for i in range(0,n)]
    k=0
    while True:
        #Check whether there is a choice final coordinate to
        #put vector in CM lattice. If so, yield that vector.
        t= sum([sigma[i]*v[i] for i in range(0,n-1)])
        if t%sigma[-1]==0:
            v[-1]=-t//sigma[-1]
            if v[-1]>=-vmax[-1] and v[-1]<=vmax[-1]:
                yield v
        #Now increment v.
        while k<n-1 and v[k]+1>vmax[k]:
            v[k]=-vmax[k]
            k=k+1
        if k<n-1:
            v[k]+=1
            k=0
        else:
            break
    
def list_irreducibles(sigma, quick=False):
    ##generate a list of irreducible elements of a CM lattice.
    #This version uses the generator function irred_generator.

    p=norm(sigma)
    n=len(sigma)
    irred_list=[]

    #Vector defining the bounds on the search space
    vmax=irred_bounds(sigma)
    if quick:
        vmax=[min(2, vmax[i]) for i in range(0,n)]
    
    for v in irred_generator(sigma,vmax):
        v_irred_flag=True
        for vtest in irred_list:
            if irred_check(v,vtest)==False:
                v_irred_flag=False
                break
        #if v is potentially irreducible, we use it filter out any
        #non-irreducibles in irred_list and add it to irred_list.
        if v_irred_flag:
            irred_list=[w for w in irred_list if irred_check(w,v)]
            irred_list+=[list(v)]

    return irred_list                    

def OSB_search(sigma, quick=False, verbose=False):
    ##Search for an obtuse superbase. Function returns an OSB
    #if it finds one. Returns false otherwise. The optional flag quick
    ##can be used to accelerate the search. When true the flag restricts
    ##the search of irreducibles to vectors with coefficients at most 2.
    ##Conjecturally, this is should still always find an OSB if it exists.
    ##
    ##This implementation is improved
    ## adding norm 2 vertices to the basis at the start

    ##

    if verbose:
        print("------")
        if quick:
            print("Accelerated search.")
        else:
            print("Standard search.")
        print("sigma="+ str(sigma))
        
        
    start_time = time.time()
    p=norm(sigma)
    n=len(sigma)

    #Create a list of norm 2 vectors that we will put in our basis.
    norm2_vertices=[]
    for i in range(1,n):
        if sigma[i]==sigma[i-1]:
            z=[0]*n
            z[i]=1
            z[i-1]=-1
            norm2_vertices+=[list(z)]

    #Filter our set of irreducibles by picking those with norm >2 and
    #pairing 0 or -1 with the norm two vectors
    vertex_pool=[]
    for z in list_irreducibles(sigma,quick):
        if len(norm2_vertices)==0 and norm(z)>0:
            vertex_pool+=[z]
        elif norm(z)>2:
            t=[sum([w[i]*z[i] for i in range(0,n)]) for w in norm2_vertices]
            if min(t)>=-1 and max(t)<=0:
                vertex_pool+=[z]

    #Now consider all possible combinations of vertices.
    #Uses combinations from itertools package.
    for t in combinations(vertex_pool,n-len(norm2_vertices)-1):
        vertex_list=norm2_vertices+list(t)
        M=np.asarray(vertex_list)
        
        #Calculate intersection matrix
        Q=np.matmul(M,M.transpose())
        
        #Check that the obtuse conditions are satisfied.
        obtuse_flag=True
        if min(sum(Q))<0:
            obtuse_flag =False
        
        for i in range(0,n-1):
            for j in range(0,i):
                if Q[i,j]>0:
                    obtuse_flag=False
                    break
        
        #Check that this is a basis by computing determinant of Q.
        #The det function in numpy doesn't seem to do integer
        #calculations.
        det=np.linalg.det(Q)
        if obtuse_flag and det>p-0.5 and det<p+0.5:
            if verbose:
                print(vertex_list)
                print(Q)
                print(det)
                print("--- Time taken: %s seconds ---" % (time.time() - start_time))
            return vertex_list

    if verbose:
        print("No OSB found.")
        print("--- Time taken: %s seconds ---" % (time.time() - start_time))
    return False

def admits_alt_surg(stable_coefs):
    ##Input the stable coefficients. Return True if
    ##a CM lattice with these coefficients has an OSB
    ##returns False otherwise.
    start_time = time.time()
    
    stable=sorted(stable_coefs)
    print(stable)

    for sigma in [[1]*i+[1]*(stable[0]-1)+stable for i in range(0,3)]:
        #Check that each sigma satisfies CM condition
        n=norm(sigma)
        CM_flag=True
        for i in range(1,len(sigma)):
            if sigma[i]> 1+sum([sigma[j] for j in range(0,i)]):
                CM_flag=False

        n=norm(sigma)
        if CM_flag:
            if OSB_search(sigma)!=False:
                print(str(n)+"-CM lattice has OSB")
            else:
                print(str(n)+"-CM lattice has no OSB")
    print("--- Time taken: %s seconds ---" % (time.time() - start_time))

            
#admits_alt_surg([2,2,2,3,6])
#-----
#Benchmark set 1
#sigma=[1,1,3,3,4,5]
#sigma=[1,1,1,3,3,4,5]
#sigma=[1,1,1,1,3,3,4,5]
#-----
#Benchmark set 2
#sigma=[1,2,2,2,3,6]
#sigma=[1,1,2,2,2,3,6]
#sigma=[1,1,1,2,2,2,3,6]
#-----
#sigma=[1,1,1,2,2,6]
#OSB_search(sigma, verbose=True)
#OSB_search(sigma,quick=True, verbose=True)

##This is code that will search for an obtuse superbase
##in an integer CM lattice. Input is a list of
##coefficients for CM lattice.

from itertools import combinations
import numpy as np
import time
import math

def is_tight(sigma):
    #Check whether a CM lattice has a tight index.
    for i in range(1,len(sigma)):
        if sigma[i]==1+ sum([sigma[j] for j in range(0,i)]):
            return True
    return False

def is_CM(sigma):
    ##Check the vector satisfies the CM condition
    for i in range(1,len(sigma)):
        if sigma[i]> 1+sum([sigma[j] for j in range(0,i)]):
            return False
    return True

def stable_is_decomposable(stable_coefs):
    #Check whether any changemaker lattice associated to a set of stable
    #coefficients can possibly be decomposable
    stable=sorted(stable_coefs)
    
    return is_decomposable([1]*stable[0]+stable)

def is_decomposable(sigma):
    #Check whether a given changemaker lattice
    #is decomposable.
    ##Uses Greene's algorithm of calculating the standard
    ##basis elements and checking that their intersection
    ##graph is connected.
    if is_tight(sigma) or not is_CM(sigma):
        return False

    STD_basis=[]
    n=len(sigma)
    for i in range(1,n):
        v=[0]*n
        v[i]=-1
        T=sigma[i]
        j=i
        while T>0:
            j=max([k for k in range(0,j) if sigma[k]<=T])
            v[j]=1
            T-=sigma[j]
        STD_basis.append(list(v))
    
    S1=[STD_basis[0]]
    S2=[]
    for i in range(1,len(STD_basis)):
        v=STD_basis[i]
        flag=True
        for w in S1:
            if sum([v[j]*w[j] for j in range(0,n)])!=0:
                flag=False
                S1.append(list(v))
                break
        if flag:
            S2.append(list(v))
            
    if len(S2)==0:
        return False
    for v in S2:
        for w in S1:
            if sum([v[j]*w[j] for j in range(0,n)])!=0:
                return False
    return True

def norm(v):
    return sum([z*z for z in v])

def irred_check(v,vtest):
    #Computes T=(v-vtest).vtest returns true if this does not prove
    #that v is reducible. I.e returns true if T<0 or vtest=0 or v=vtest.

    T=sum([(v[i]-vtest[i])*vtest[i] for i in range(0,len(v))])
    if T<0:
        return True

    if T==0:
        if sum([z*z for z in vtest])==0:
            return True
        if sum([(v[i]-vtest[i])*(v[i]-vtest[i]) for i in range(0,len(v))])==0:
            return True
    return False
    
def vertex_bounds(sigma):
    #Provide bounds on the coefficients of a vertex in
    #in an OSB in a changemaker lattice.
    n=len(sigma)
    vmax=[0]*n
    p=norm(sigma)
    #Check if sigma is tight
    vmax[0]=1
    if is_tight(sigma):
        vmax[0]=2
    else:
        vmax[0]=1
    
    for i in range(1, len(sigma)):
        if sigma[i]==sigma[i-1]:
            vmax[i]=vmax[i-1]
            continue

        if sigma[i-1]==1 and sigma[i]>1:
            vmax[i]=sigma[i]
            continue   
        
        #If sigma[i] tight.
        if sigma[i]> sum([sigma[j] for j in range(0,i)]):
            vmax[i]=5+sum([vmax[j]+1 for j in range(0,i)])
        else:
            T=sigma[i]
            j=i-1
            while T>0:
                j=max([k for k in range(0,j+1) if sigma[k]<=T])
                vmax[i]=vmax[i]+vmax[j]+1
                T-=sigma[j]
    #Compute the bound coming from Cauchy-Schwartz inequality
    for i in range(1,len(sigma)):
        CS_bound= int(math.sqrt(p - sigma[i]*sigma[i]))
        vmax[i]=min(CS_bound, vmax[i])
    
    return vmax

def CM_generator(sigma, vmax):
    ##Iterates through all vectors in a changemaker lattice
    ##whose coordinates satisfy |v[i]|\leq vmax[i] for all i.
    n=len(sigma)
    
    v=[-vmax[i] for i in range(0,n)]
    k=0
    while True:
        #Check whether there is a choice final coordinate to
        #put vector in CM lattice. If so, yield that vector.
        t= sum([sigma[i]*v[i] for i in range(0,n-1)])
        if t%sigma[-1]==0:
            v[-1]=-t//sigma[-1]
            if v[-1]>=-vmax[-1] and v[-1]<=vmax[-1]:
                yield v
        #Now increment v.
        while k<n-1 and v[k]+1>vmax[k]:
            v[k]=-vmax[k]
            k=k+1
        if k<n-1:
            v[k]+=1
            k=0
        else:
            break
    
def list_vertices(sigma, quick=False):
    ##generate a list of candidate vertices with norm at least 3.
    ##If the quick flag is set to true, this does not
    ##generate precisely the set of all irreducibles.
    p=norm(sigma)
    n=len(sigma)
    irred_list=[]
    equal_list=[]
    
    #Vector defining the bounds on the search space
    vmax=vertex_bounds(sigma)

    #Modify vmax if we are doing a quick search
    if quick:
        vmax=[min(2, vmax[i]) for i in range(0,n)]
        for i in range(0,n):
            if sigma[i]==sigma[-1]:
                vmax[i]=1
    
    for k in range(1,sigma[-1]+1):
        if sigma.count(k)>1:
            equal_list.append([i for i in range(0,n) if sigma[i]==k])
    
    for v in CM_generator(sigma,vmax):
        #Skip vectors of norm  less than 2.
        if norm(v)<=2:# or norm_calc>p:
            continue
        v_irred_flag=True
        #Check whether there is a vertex of norm 2 which
        #shows reducibility
        for clist in equal_list:
            mini=min([v[i] for i in clist])
            maxi=max([v[i] for i in clist])
            if maxi>mini+1:
                v_irred_flag=False
                break
        ##Finally compare v against all other potential irreducibles.
        if v_irred_flag:
            for vtest in irred_list:
                if irred_check(v,vtest)==False:
                    v_irred_flag=False
                    break
        #if v is potentially irreducible, we use it filter out any
        #non-irreducibles in irred_list and add it to irred_list.
        if v_irred_flag:
            irred_list=[w for w in irred_list if irred_check(w,v)]
            irred_list+=[list(v)]
    return irred_list

def obtuse_check(Q,n):
    if min(sum(Q))<0:
        return False
        
    for i in range(0,n-1):
        for j in range(0,i):
            if Q[i,j]>0:
                return False
    return True
    

def OSB_search(sigma, quick=False):
    ##Search for an obtuse superbase. Function returns an OSB
    #if it finds one. Returns false otherwise. The optional flag quick
    ##can be used to accelerate the search. When true the flag restricts
    ##the search of irreducibles to vectors with coefficients at most 2.
    ##Conjecturally, this is should still always find an OSB if it exists.
    ##However, we can only rigourously rule out the existence of
    ##an OSB if we run the algorithm with the quick flag set to False.
    ##This implementation is improved
    ## adding norm 2 vertices to the basis at the start

    p=norm(sigma)
    n=len(sigma)
    
    #Create a list of norm 2 vectors that we will put in our basis.
    norm2_vertices=[]
    for i in range(1,n):
        if sigma[i]==sigma[i-1]:
            z=[0]*n
            z[i]=1
            z[i-1]=-1
            norm2_vertices+=[list(z)]

    #Filter our set of irreducibles by picking those with norm >2 and
    #pairing 0 or -1 with the norm two vectors
    vertex_pool=[]
    for z in list_vertices(sigma,quick):
        t=[sum([w[i]*z[i] for i in range(0,n)]) for w in norm2_vertices]
        if len(t)==0 or (min(t)>=-1 and max(t)<=0):
            vertex_pool+=[z]

    #Now consider all possible combinations of vertices.
    #Uses combinations from itertools package.
    for t in combinations(vertex_pool,n-len(norm2_vertices)-1):
        vertex_list=norm2_vertices+list(t)
        M=np.asarray(vertex_list)
        
        #Calculate intersection matrix
        Q=np.matmul(M,M.transpose())
        
        #Check that this is a basis by computing determinant of Q.
        #The det function in numpy doesn't seem to do integer
        #calculations.
        det=np.linalg.det(Q)
        if obtuse_check(Q,n) and det>p-0.5 and det<p+0.5:
            return Q
    return []

def adhoc_tests(stable_coefs):
    ##Some tests for whether a given set of cable coefficients
    ##can possibly support an OSB. Returns false if there cannot
    ##exist an OSB. Returns true if inconclusive. These methods
    ##are not necessary they are simply used to accelerate running
    ##times.
    stable=sorted(stable_coefs)
    ##If the stable coefficents contain 4,4,4,4,3,3,2 then
    ##there is no OSB.
    if stable.count(4)>=4 and stable.count(3)>=2 and stable.count(2)>=1:
        return False

    ##If the stable coefficents contain 3,3,3,3,2,2,2 then
    ##there is no OSB.
    if stable.count(3)>=4 and stable.count(2)>=3:
        return False
    ##The obstruction based on Corollary ?? from the paper.
    for k in range(0,len(stable)-1):
        N=stable.count(stable[k])
        if N>=4 and (N-1)*stable[k]>stable[k+1]>stable[k]:
            return False

    return True

def find_OSB_slopes(stable_coefs):
    ##Input the stable coefficients. Returns a rigourously verified
    ##list of coefficients for which there is an OSB along
    ##with a gram matrix for each OSB.
    output=[]
    if adhoc_tests(stable_coefs)==False:
        return output
    
    stable=sorted(stable_coefs)

    ##Check whether these stable coefficients support a decomposable lattice.
    decomposable= stable_is_decomposable(stable)

    slopes=[]
    ##Work out which slopes can conceivably admit an OSB.
    if decomposable:
        slopes=[0,1,2]
    elif is_CM([1]*(stable[0]-1)+stable):
        slopes=[0,1]
    else:
        slopes=[1,2]
    
    #First perform a quick search to find OSBs with small coefficients.
    for k in slopes:
        sigma=[1]*k+[1]*(stable[0]-1)+stable
        n=norm(sigma)
        
        if is_CM(sigma):
            ##First run a quick search
            result=OSB_search(sigma,quick=True)
            if len(result)>0:
                output+=[[n,result]]
            else:
                ##If an OSB not found quickly, run a full search instead.
                result=OSB_search(sigma, quick=False)
                if len(result)>0:
                    output+=[[n,result]]
            
    return output


In [2]:
stable_coefficients=[['BL_knot:[1, 1, 1, 1, 0]', [12, 9, 5, 4, 2]],
 ['BL_knot:[1, 1, 0, 1, 1]', [16, 12, 7, 4, 2]],
 ['BL_knot:[1, 1, 1, 2, 0]', [12, 12, 9, 5, 4, 2]],
 ['BL_knot:[1, 1, 2, 1, 0]', [17, 14, 5, 5, 4, 2]],
 ['BL_knot:[1, 2, 1, 1, 0]', [17, 13, 7, 6, 3]],
 ['BL_knot:[2, 1, 1, 1, 0]', [18, 13, 7, 6, 2, 2]],
 ['BL_knot:[1, 1, 1, 1, 1]', [26, 17, 12, 5, 4, 2]],
 ['BL_knot:[1, 1, 0, 2, 1]', [23, 19, 7, 7, 4, 2]],
 ['BL_knot:[1, 2, 0, 1, 1]', [23, 17, 10, 6, 3]],
 ['BL_knot:[1, 1, 2, 2, 0]', [17, 17, 14, 5, 5, 4, 2]],
 ['BL_knot:[1, 2, 1, 2, 0]', [17, 17, 13, 7, 6, 3]],
 ['BL_knot:[2, 1, 1, 2, 0]', [18, 18, 13, 7, 6, 2, 2]],
 ['BL_knot:[1, 1, 0, 1, 2]', [28, 12, 12, 7, 4, 2]],
 ['BL_knot:[2, 1, 0, 1, 1]', [24, 18, 11, 6, 2, 2]]]

In [3]:
start_time = time.time()
GOERITZ=[]

for [knot,stable] in stable_coefficients:
    goeritz=find_OSB_slopes(stable)
    print(knot)
    print(goeritz)
    GOERITZ.append([knot, stable,goeritz])

print("--- Time taken: %s seconds ---" % ((time.time() - start_time)))

BL_knot:[1, 1, 1, 1, 0]
[[271, array([[ 4,  0, -2,  0, -1],
       [ 0,  3,  0,  0, -2],
       [-2,  0,  4, -1, -1],
       [ 0,  0, -1,  3, -1],
       [-1, -2, -1, -1,  6]])], [272, array([[ 2,  0,  0,  0, -1,  0],
       [ 0,  4,  0, -1, -1, -1],
       [ 0,  0,  3,  0,  0, -2],
       [ 0, -1,  0,  3, -1, -1],
       [-1, -1,  0, -1,  3,  0],
       [ 0, -1, -2, -1,  0,  5]])]]
BL_knot:[1, 1, 0, 1, 1]
[[470, array([[ 5,  0, -1, -1, -2],
       [ 0,  4, -1, -1, -2],
       [-1, -1,  5, -1, -1],
       [-1, -1, -1,  4,  0],
       [-2, -2, -1,  0,  5]])], [471, array([[ 2,  0, -1,  0,  0,  0],
       [ 0,  3, -1,  0, -1, -1],
       [-1, -1,  4,  0, -1, -1],
       [ 0,  0,  0,  3, -1, -1],
       [ 0, -1, -1, -1,  4, -1],
       [ 0, -1, -1, -1, -1,  6]])]]
BL_knot:[1, 1, 1, 2, 0]
[[415, array([[ 2,  0,  0,  0,  0, -1],
       [ 0,  6, -1, -1, -2, -1],
       [ 0, -1,  3, -1,  0,  0],
       [ 0, -1, -1,  4,  0, -2],
       [ 0, -2,  0,  0,  3,  0],
       [-1, -1,  0, -2,  0,  4]]

In [4]:
GOERITZ

[['BL_knot:[1, 1, 1, 1, 0]',
  [12, 9, 5, 4, 2],
  [[271,
    array([[ 4,  0, -2,  0, -1],
           [ 0,  3,  0,  0, -2],
           [-2,  0,  4, -1, -1],
           [ 0,  0, -1,  3, -1],
           [-1, -2, -1, -1,  6]])],
   [272,
    array([[ 2,  0,  0,  0, -1,  0],
           [ 0,  4,  0, -1, -1, -1],
           [ 0,  0,  3,  0,  0, -2],
           [ 0, -1,  0,  3, -1, -1],
           [-1, -1,  0, -1,  3,  0],
           [ 0, -1, -2, -1,  0,  5]])]]],
 ['BL_knot:[1, 1, 0, 1, 1]',
  [16, 12, 7, 4, 2],
  [[470,
    array([[ 5,  0, -1, -1, -2],
           [ 0,  4, -1, -1, -2],
           [-1, -1,  5, -1, -1],
           [-1, -1, -1,  4,  0],
           [-2, -2, -1,  0,  5]])],
   [471,
    array([[ 2,  0, -1,  0,  0,  0],
           [ 0,  3, -1,  0, -1, -1],
           [-1, -1,  4,  0, -1, -1],
           [ 0,  0,  0,  3, -1, -1],
           [ 0, -1, -1, -1,  4, -1],
           [ 0, -1, -1, -1, -1,  6]])]]],
 ['BL_knot:[1, 1, 1, 2, 0]',
  [12, 12, 9, 5, 4, 2],
  [[415,
    array([[ 